In [5]:
import gzip
from pathlib import Path

import dask.dataframe as dd
import numpy as np
import pandas as pd
import pyarrow as pa
from dask.distributed import Client
from dask.distributed import LocalCluster
from rdkit import Chem
from rdkit import RDLogger
from rdkit.Chem import rdMolTransforms

from confscale.metadata import get_etkdg_info

In [ ]:
# Create a local cluster with 18 workers
cluster = LocalCluster(n_workers=18)

# Create a client connected to the cluster
client = Client(cluster)

# Display client information
client

/home/etienne-reboul/anaconda3/envs/confscale-dev/lib/python3.13/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 32947 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:32947/status,
Dashboard: http://127.0.0.1:32947/status,Workers: 18
Total threads: 36,Total memory: 31.03 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37663,Workers: 0
Dashboard: http://127.0.0.1:32947/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:46641,Total threads: 2
Dashboard: http://127.0.0.1:34097/status,Memory: 1.72 GiB
Nanny: tcp://127.0.0.1:35233,


2025-08-29 14:09:37,134 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:43809' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('toparquetdata-70c5137c443e1448cf433f2fd3d6a3bb', 880), ('frompandas-0d567b8bcc5ebfe7a7caddeeeff72a57', 817), ('toparquetdata-70c5137c443e1448cf433f2fd3d6a3bb', 943), ('partition_wrapper-8cec55ca6d10cf4a70e6d3c987e62f95', 840), ('toparquetdata-70c5137c443e1448cf433f2fd3d6a3bb', 933), ('toparquetdata-70c5137c443e1448cf433f2fd3d6a3bb', 901), ('toparquetdata-70c5137c443e1448cf433f2fd3d6a3bb', 997), ('toparquetdata-70c5137c443e1448cf433f2fd3d6a3bb', 881), ('toparquetdata-70c5137c443e1448cf433f2fd3d6a3bb', 84), ('toparquetdata-70c5137c443e1448cf433f2fd3d6a3bb', 90), ('toparquetdata-70c5137c443e1448cf433f2fd3d6a3bb', 980), ('toparquetdata-70c5137c443e1448cf433f2fd3d6a3bb', 964), ('frompandas-0d567b8bcc5ebfe7a7caddeeeff72a57', 818)} (stimulus_id='handle-worker-cleanup-1756469377.134159')
2025-08-29 14:09:

In [7]:
def compressedsdf2mols(filepath: str | Path) -> list[Chem.Mol]:
    """
    Read molecules from a compressed SDF file.

    Parameters
    ----------
    filepath : str
       Path to the compressed SDF file (.sdf.gz)

    Returns
    -------
    list[Chem.Mol]
       List of RDKit molecule objects loaded from the file,
       excluding any None entries (failed molecule parsing)
    """
    # initiate
    mols = []
    # Set the RDKit logger to only show critical messages
    RDLogger.logger().setLevel(RDLogger.CRITICAL)
    with gzip.open(filepath) as gzf:
        suppl = Chem.ForwardSDMolSupplier(gzf)
        mols = [mol for mol in suppl if mol is not None]
    return mols

In [8]:
def process_molecule(mol: Chem.Mol) -> list[dict]:
    """
    Process a molecule to extract dihedral angle information.

    Parameters
    ----------
    mol : Chem.Mol
        RDKit molecule object with 3D coordinates.

    Returns
    -------
    list[dict]
        List of dictionaries containing dihedral angle information with keys:
        - identifier (string): CSD identifier of the molecule
        - smarts (string): SMARTS pattern identifying the rotatable bond
        - angle (np.float32): Dihedral angle in degrees (0-360)

    Notes
    -----
    This function extracts torsion angles from the molecule using ETKDG information.
    Angles are normalized to the range [0, 360] degrees.
    """
    # init
    results = []

    # get torsin angles infos
    try:
        infos = get_etkdg_info(mol)
    except Exception as e:
        print(f"Error getting ETKDG info: {e}")
        return results

    # get CSD identifier
    identifier = mol.GetProp("CSD_Entry_Name") if mol.HasProp("CSD_Entry_Name") else None

    # iterate on the infos

    for angle_info in infos:
        # loop variable
        temp_dict = {
            "identifier": identifier,
            "smarts": angle_info["smarts"],
            "angle": np.float32(rdMolTransforms.GetDihedralDeg(mol.GetConformer(), *angle_info["atomIndices"])),
        }
        temp_dict["angle"] = temp_dict["angle"] + 360 if temp_dict["angle"] < 0 else temp_dict["angle"]
        results.append(temp_dict)

    return results


def partition_wrapper(partition: pd.DataFrame) -> pd.DataFrame:
    """
    Process a partition of a Dask DataFrame containing paths to SDF files.

    Parameters
    ----------
    partition : pd.DataFrame
        DataFrame partition with a 'filepath' column containing paths to compressed SDF files.

    Returns
    -------
    pd.DataFrame
        DataFrame containing extracted dihedral angle information with columns:
        - identifier (str): CSD identifier of the molecule
        - smarts (str): SMARTS pattern identifying the rotatable bond
        - angle (float32): Dihedral angle in degrees normalized to [0, 360]

    Notes
    -----
    For this function to work as intended the partition should be a single row DataFrame.
    This function processes each molecule in the SDF file to extract dihedral angles
    using the process_molecule function. It's designed to be used with Dask's map_partitions.
    """

    # instantiate variables
    results = []

    #  retrieve path to compressed SDF files
    sdf_gz_path = partition["filepath"].values[0]

    # read the compressed SDF file
    mols = compressedsdf2mols(sdf_gz_path)

    # process the molecules
    for mol in mols:
        temp_results = process_molecule(mol)
        if temp_results:
            results.extend(temp_results)

    # cast as DataFrame
    df = pd.DataFrame(results)

    return df

In [4]:
datapath = Path("../data/processed/csd_sdf_batches/")
schema = pa.schema(
    [("identifier", pa.string()), ("smarts", pa.string()), ("angle", pa.float32())],
    metadata={
        "description": "Dataset containing molecular dihedral angle information",
        "identifier_description": "CSD identifier of the molecule",
        "smarts_description": "SMARTS patterns identifying rotatable bonds",
        "angle_description": "Dihedral angle in degrees (0-360)",
        "source": "CSD dataset processed with RDKit",
        "creator": "confscale pipeline",
    },
)

In [9]:
df = pd.DataFrame(datapath.glob("*.sdf.gz"), columns=["filepath"])

In [10]:
ddf = dd.from_pandas(df, npartitions=df.shape[0])
ddf = ddf.map_partitions(partition_wrapper, meta={"identifier": "str", "smarts": "str", "angle": "float32"})
ddf.to_parquet("../data/processed/csd_dihedral_angles", schema=schema)

In [11]:
results_df = pd.concat([pd.read_parquet(path) for path in Path("../data/processed/csd_dihedral_angles").glob("*.parquet")])

In [12]:
results_df.to_parquet(datapath / "csd_dehydrals.parquet")